# 1.Imports

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import gzip
import numpy as np
import sys

# TODO : use random seed

# 2. Loading and preparation of data
As a basis for comparison we will be using the MNIST dataset. If we manage to do all the work we want, we will then use other datasets for comparison.

### 2.1. Definition of methods to extract data and labels

In [2]:
def extract_data(filename, image_shape, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(np.prod(image_shape) * image_number)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(image_number, image_shape[0], image_shape[1])
    return data


def extract_labels(filename, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * image_number)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return labels

### 2.2. Load data

In [3]:
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000
data_folder = 'mnist_data'

train_images_path = os.path.join(data_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_folder, 't10k-labels-idx1-ubyte.gz')

train_images = extract_data(train_images_path, image_shape, train_set_size)
test_images = extract_data(test_images_path, image_shape, test_set_size)
train_labels = extract_labels(train_labels_path, train_set_size)
test_labels = extract_labels(test_labels_path, test_set_size)

### 2.3. Convert data from numpy arrays to torch tensors

In [4]:
features_train=torch.from_numpy(train_images)
features_test=torch.from_numpy(test_images)
print('Training features:', features_train.shape, '\n'
'Testing features:', features_test.shape)

labels_train=torch.from_numpy(train_labels)
labels_test=torch.from_numpy(test_labels)
print('Training labels:', labels_train.shape, '\n'
'Testing labels:', labels_test.shape)

Training features: torch.Size([60000, 28, 28]) 
Testing features: torch.Size([10000, 28, 28])
Training labels: torch.Size([60000]) 
Testing labels: torch.Size([10000])


### 2.4. Normalize data

In [5]:
mean, std = features_train.float().mean(), features_train.float().std()

features_train = features_train.float().sub_(mean).div_(std)
features_test = features_test.float().sub_(mean).div_(std)

# 3. Setting up network and evaluation methods

### 3.1. Multilayer perceptron (MLP)

##### 3.1.1. Defining class

In [6]:
class MLP(nn.Module):
    
    def __init__(self, hidden_size_1=512, hidden_size_2=100, hidden_size_3=10):
        super(MLP, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, hidden_size_1),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size_1, hidden_size_2))
            # nn.ReLU(inplace=True),
            # nn.Linear(hidden_size_2, hidden_size_3)) #maybe try more layers
    
    # forward pass
    def forward(self, x):
        return self.layers(x)

mlp = MLP()

##### 3.1.2. Implementation of method for training

In [64]:
def mlp_nn(x_train, y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch):
    loss_all_train, loss_all_test = [], []
    loss_train_ret = 0
    loss_test_ret = 0
    epochs_all = torch.arange(1, num_epoch+num_epoch/10, num_epoch/10)
    epochs_all[-1] = num_epoch - 1
            
    for epoch in range(num_epoch):
        for b in range(0, x_train.size(0), size_minibatch):
            y = model(x_train[b:b+size_minibatch])
            loss_train = criterion(y, y_train[b:b+size_minibatch])
        
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
        
        if epoch % 20 == 0:
            loss_train = loss_train.detach().numpy()
            loss_all_train.append(loss_train)

            y_test_obt = model(x_test)
            loss_test = criterion(y_test_obt, y_test)
            loss_test = loss_test.detach().numpy()
            loss_all_test.append(loss_test)
            
            print ('Epoch [%d/%d], Train Loss: %.4f, Test Loss: %.4f' %(epoch+1, num_epoch, loss_train, loss_test))

        if epoch == num_epoch - 1:
            loss_train = loss_train.detach().numpy()
            loss_all_train.append(loss_train)

            y_test_obt = model(x_test)
            loss_test = criterion(y_test_obt, y_test)
            loss_test = loss_test.detach().numpy()
            loss_all_test.append(loss_test)
            
            loss_train_ret = loss_train
            loss_test_ret = loss_test
            
            print('Final, Train Loss: %.4f, Test Loss: %.4f' %(loss_train, loss_test))

            # plotting train and test loss

            #fig, axs = plt.subplots(1, 2, figsize=(10, 8))
            #axs[0].plot(epochs_all, loss_all_train, linewidth=2.5, color='blue')
            #axs[0].set_ylabel('Train Loss', fontsize=12)
            #axs[0].set_xlabel('Epoch', fontsize=12)

            #axs[1].plot(epochs_all, loss_all_test, linewidth=2.5, color='orange')
            #axs[1].set_ylabel('Test Loss', fontsize=12)
            #axs[1].set_xlabel('Epoch', fontsize=12)
            #fig.suptitle('MLP Convergence', fontsize=15)
            #plt.show()

    return loss_train_ret, loss_test_ret

# 5. Metrics of our tuning protocol
At this stage, we want to select the hyperparameter search space for each optimizer. This way, we can first tune the hyperparameters of each optimizer separately and then select the trial that achieved lowest final validation error.
We then comapre the optimizers' performance by looking at the validation and test errors as suggested in the paper "On empirical comparisons of optimizers for deep learning".

We will also look at the training speed (number of training steps required) to reach a traget validation error.

Everything is tuned on a log scale.

No L_2 regularization or weight decay is used.

### 5.1. Tuning protocol using bootstrap
To estimate means and uncertainties of our tuning protocol we will use bootstrapping starting from an initial search space suggested by the paper "On Empirical Comparisons of Optimizers for Deep Learning".
We run N trials by randomly picking values in the search space of the algorithm at every trial.
Then we sample these trials with replacement and compute our statistic on the first K trials of this sample. We repeat this process 100 times and compute the 5th percentile and 95th percentile of the bootstrap distribution.

This allows us to plot the error bars to show the results.

### 5.2. Tuning Adam for a CNN on MNIST
The hyperparameters we are tuning are alpha_0/epsilon, 1 - beta_1, 1 - beta_2, epsilon.
The initial search spaces are suggested based on the experience of the writers of the same paper, "On empirical comparisons of optimizers for deep learning".
N is also suggested to be 500 and K to be 100.

##### 5.2.1. Set up parameters and search space

In [54]:
# TODO: We used the final search spaces instead of the initial ones, should we reproduce the whole method

N = 500 # Number of trials
K = 100 # Number of trials being kept for the statistic

# Search spaces for parameters
alpha_0 = np.linspace(10**(-1), 10, N)
beta_1 = np.linspace(10**(-3), 1, N)
beta_2 = np.linspace(10**(-4), 1, N)
eps = np.linspace(10**(-6), 10**(-2), N)

# TODO: tune number of decay steps between 0.5 and 1 times the number of training steps
# TODO : tune learning rate decay factor within 10**-3, 10**-2, 10**-1

##### 5.2.2. Set up model for training

In [55]:
# Model fixed parameters
model = MLP()
criterion = nn.CrossEntropyLoss() # good loss function for classification tasks
num_epoch = 50
size_minibatch = 128

x_train = features_train
y_train = labels_train
x_test = features_test
y_test = labels_test

##### 5.2.2. Estimating trail outcomes via bootstrap

In [ ]:
# We first run and store N trials
N_trials = []

for _ in range(N):
    # Pick random values from the intervals given for the different parameters
    alpha_0_pick  = float(np.random.choice(alpha_0, 1)) # np.random.choice samples uniformely with replacement
    beta_1_pick   = float(-np.random.choice(beta_1, 1) + 1)
    beta_2_pick   = float(-np.random.choice(beta_2, 1) + 1)
    eps_pick      = float(np.random.choice(eps, 1))
    learning_rate = alpha_0_pick * eps_pick
    
    # Build optimizer from parameters
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta_1_pick, beta_2_pick), eps=eps_pick)
    
    # Run
    train_error, test_error = mlp_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch)
    
    # Store parameters, train and test error
    N_trials.append([beta_1_pick, beta_2_pick, eps_pick, learning_rate, train_error, test_error])

    
# TODO : finish this shit

# Do the following 100 times :
for _ in range(100):
    # Resample N samples from the N-trials with replacement
    N_sampled = np.random.choice(N_trials, N)
    
    # Compute statistic on the first K trials of the resampled dataset

    
# Compute means, 5th percentile, 95 percentile of bootrap distribution
# Generate means and error bars for each plot

##### 5.2.3. Tune to find best parameter
We perform trials until we have K of them, then we pick the best based on our statistic of interest

In [ ]:
nb_hyperamaters_to_tune = 4
nb_exported_statistics  = 2

lowest_test_error = [sys.maxsize] * (nb_hyperamaters_to_tune + nb_exported_statistics)



for _ in range(K):
    # Pick random values from the intervals given for the different parameters
    alpha_0_pick  = float(np.random.choice(alpha_0, 1)) # np.random.choice samples uniformely with replacement
    beta_1_pick   = float(-np.random.choice(beta_1, 1) + 1)
    beta_2_pick   = float(-np.random.choice(beta_2, 1) + 1)
    eps_pick      = float(np.random.choice(eps, 1))
    learning_rate = alpha_0_pick * eps_pick
    
    # Build optimizer from parameters
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta_1_pick, beta_2_pick), eps=eps_pick)
    
    # Run
    train_error, test_error = mlp_nn(x_train,y_train, x_test, y_test, model, optimizer, criterion, num_epoch, size_minibatch)
    
    # Concatenate hyperparameters with results
    vector = [beta_1_pick, beta_2_pick, eps_pick, learning_rate, train_error, test_error]
    
    # Check wether we have the smallest test error and store parameters in case we find it
    if test_error < lowest_test_error[len(lowest_test_error) - 1]:
        lowest_test_error = vector

Epoch [1/50], Train Loss: 0.0000, Test Loss: 0.1904
Epoch [21/50], Train Loss: 0.0000, Test Loss: 0.1915
Epoch [41/50], Train Loss: 0.0000, Test Loss: 0.1924
Final, Train Loss: 0.0000, Test Loss: 0.1927
Epoch [1/50], Train Loss: 0.0000, Test Loss: 0.1928
Epoch [21/50], Train Loss: 0.0000, Test Loss: 0.1929
Epoch [41/50], Train Loss: 0.0000, Test Loss: 0.1931
Final, Train Loss: 0.0000, Test Loss: 0.1931
Epoch [1/50], Train Loss: 0.0000, Test Loss: 0.1931
Epoch [21/50], Train Loss: 0.0000, Test Loss: 0.1937
Epoch [41/50], Train Loss: 0.0000, Test Loss: 0.1942
Final, Train Loss: 0.0000, Test Loss: 0.1945
Epoch [1/50], Train Loss: 0.0000, Test Loss: 0.1945
Epoch [21/50], Train Loss: 0.0000, Test Loss: 0.1950
Epoch [41/50], Train Loss: 0.0000, Test Loss: 0.1954
Final, Train Loss: 0.0000, Test Loss: 0.1956
Epoch [1/50], Train Loss: 0.0000, Test Loss: 0.1956
Epoch [21/50], Train Loss: 0.0000, Test Loss: 0.1956
Epoch [41/50], Train Loss: 0.0000, Test Loss: 0.1956
Final, Train Loss: 0.0000, Tes

In [ ]:
# Print best parameters

print('Beta 1: %.2f' % lowest_test_error[0])
print('Beta 2: %.2f' % lowest_test_error[1])
print('Epsilon: %.2f' % lowest_test_error[2])
print('Learning rate: %.2f' % lowest_test_error[3])
print('Train error: %.2f' % lowest_test_error[4])
print('Test error: %.2f' % lowest_test_error[5])